In [0]:

%fs
ls /Volumes/workspace/ecommerce/ecommerce_data

In [0]:

df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_oct.limit(5).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5001201392501755>, line 2
      1 df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
----> 2 df_oct.limit(5).show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
    865             },
    866         )
    868 table, _ = DataFra

In [0]:

from pyspark.sql.functions import to_date, col

silver_df = spark.table("ecommerce.silver.events")

# Add derived date column
silver_df = silver_df.withColumn("event_date",to_date(col("event_time")))

# Overwrite with schema evolution enabled
silver_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").saveAsTable("ecommerce.silver.events")

In [0]:
silver_df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .partitionBy("event_date") \
    .saveAsTable("ecommerce.silver.events")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5001201392501757>, line 5
      1 silver_df.write.format("delta") \
      2     .mode("overwrite") \
      3     .option("mergeSchema", "true") \
      4     .partitionBy("event_date") \
----> 5     .saveAsTable("ecommerce.silver.events")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observati

In [0]:

%sql
DROP TABLE ecommerce.silver.events;

In [0]:

from pyspark.sql.functions import to_date, col

silver_df = spark.table("workspace.ecommerce.silver_events")

silver_df = silver_df.withColumn(
    "event_date",
    to_date(col("event_time"))
)

silver_df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("event_date") \
    .saveAsTable("ecommerce.silver.events")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5001201392501759>, line 13
      3 silver_df = spark.table("workspace.ecommerce.silver_events")
      5 silver_df = silver_df.withColumn(
      6     "event_date",
      7     to_date(col("event_time"))
      8 )
     10 silver_df.write.format("delta") \
     11     .mode("overwrite") \
     12     .partitionBy("event_date") \
---> 13     .saveAsTable("ecommerce.silver.events")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:737, in DataFrameWriter.saveAsTable(self, name, format, mode, partitionBy, **options)
    735 self._write.table_name = name
    736 self._write.table_save_method = "save_as_table"
--> 737 _, _, ei = self._spark.client.execute_command(
    738     self._write.command(self._spark.client), self._write.observations
    739 )
    740 self._callback(ei)

File /databric

In [0]:

%sql
OPTIMIZE ecommerce.silver.events
ZORDER BY (event_type, category_code, user_id);